# Liquid Neural Networks for Hydrological Inflow Forecasting

This notebook presents a comprehensive analysis of Liquid Neural Networks (LNNs) for hydrological time series forecasting. We compare the performance of LNNs against traditional recurrent neural networks (RNNs) like LSTM and GRU, as well as 1D-CNNs, on a real-world rainfall-inflow dataset.

The primary objectives of this study are:
1. To evaluate the effectiveness of LNNs in a hydrological context.
2. To benchmark LNN performance against established deep learning models.
3. To explore advanced LNN capabilities, such as handling irregular time series and providing uncertainty estimates.

This notebook is structured to be a reproducible research artifact, including detailed explanations, code, and a series of 10 structured experiments to thoroughly investigate the properties of LNNs.

## 1. Setup and Imports

First, we import the necessary libraries for data manipulation, modeling, and visualization.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchdiffeq import odeint
import time
import json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from IPython.display import Markdown, display

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

## 2. Data Loading and Preprocessing

We load and preprocess the dataset. This includes converting the 'Date' column, setting it as the index, and scaling all numerical features to a [0, 1] range using `MinMaxScaler`.

In [ ]:
file_path = "final_inflow_rainfall_merged.csv"
df = pd.read_csv(file_path)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df.values)

df_scaled = pd.DataFrame(data_scaled, columns=df.columns, index=df.index)

### Creating Time Series Sequences

We create sequences of data for our time series models. Each sample will consist of a window of 5 previous days (`win_length`) to predict the next day's inflow.

In [ ]:
def create_sequences(data, target_col, win_length):
    sequences = []
    labels = []
    for i in range(len(data) - win_length):
        seq = data[i:i+win_length]
        label = data[i+win_length:i+win_length+1, target_col]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

win_length = 5
target_col_index = df.columns.get_loc('Inflows in last 24 Hrs. in Mcft.')
X, y = create_sequences(data_scaled, target_col_index, win_length)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, shuffle=False)

# Convert to PyTorch Tensors
X_train_t = torch.from_numpy(X_train).float()
y_train_t = torch.from_numpy(y_train).float()
X_test_t = torch.from_numpy(X_test).float()
y_test_t = torch.from_numpy(y_test).float()

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), batch_size=batch_size, shuffle=False)

## 4. Model Implementation and Training

We define all models within the PyTorch framework for consistent comparison.

In [ ]:
def train_model(model, train_loader, epochs=50, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    return model

### 4.1. TrueLNN Model (with torchdiffeq)

In [ ]:
class ODEF(nn.Module):
    def __init__(self, hidden_size):
        super(ODEF, self).__init__()
        self.net = nn.Sequential(nn.Linear(hidden_size, hidden_size), nn.Tanh(), nn.Linear(hidden_size, hidden_size))
    def forward(self, t, x):
        return self.net(x)

class TrueLNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TrueLNN, self).__init__()
        self.hidden_size = hidden_size
        self.input_to_hidden = nn.Linear(input_size, hidden_size)
        self.ode_func = ODEF(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h_t = torch.zeros(batch_size, self.hidden_size)
        for t in range(x.size(1)):
            input_t = self.input_to_hidden(x[:, t, :])
            h_t = odeint(self.ode_func, h_t + input_t, torch.tensor([0, 1]), method='rk4')[1]
        return self.fc(h_t)

lnn_model = TrueLNN(X_train.shape[2], 128, 1)
print("Training TrueLNN...")
lnn_model = train_model(lnn_model, train_loader, epochs=50)

### 4.2. Baseline Models (LSTM, GRU, 1D-CNN) in PyTorch

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])

class CNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(CNNModel, self).__init__()
        self.conv1d = nn.Conv1d(input_size, 128, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128 * win_length, output_size)
    def forward(self, x):
        x = x.permute(0, 2, 1) # (B, F, T)
        x = self.relu(self.conv1d(x))
        x = self.flatten(x)
        return self.fc(x)

# Train baseline models
lstm_torch_model = LSTMModel(X_train.shape[2], 128, 1)
gru_torch_model = GRUModel(X_train.shape[2], 128, 1)
cnn_torch_model = CNNModel(X_train.shape[2], 1)

print("Training PyTorch LSTM...")
lstm_torch_model = train_model(lstm_torch_model, train_loader)
print("\nTraining PyTorch GRU...")
gru_torch_model = train_model(gru_torch_model, train_loader)
print("\nTraining PyTorch 1D-CNN...")
cnn_torch_model = train_model(cnn_torch_model, train_loader)

## 5. Model Evaluation and Comparison

We evaluate all models on the test set and compare their performance.

In [ ]:
results = {}
df_final = df.iloc[len(df) - len(y_test) + win_length - 1:].copy()
actual = df_final['Inflows in last 24 Hrs. in Mcft.']

# TensorFlow models
for name, model in models.items():
    preds_scaled = model.predict(test_generator)
    preds_inv = inverse_transform_predictions(preds_scaled, X_test, scaler, win_length)
    df_final[f'{name}_Pred'] = preds_inv
    mse = mean_squared_error(actual, preds_inv)
    results[name] = {'RMSE': np.sqrt(mse), 'MSE': mse, 'R-squared': r2_score(actual, preds_inv), 'MAE': mean_absolute_error(actual, preds_inv)}

# PyTorch LNN
lnn_model.eval()
with torch.no_grad():
    preds_lnn_scaled = lnn_model(X_test_t).numpy()
preds_lnn_inv = inverse_transform_predictions(preds_lnn_scaled, X_test, scaler, win_length)
df_final['LNN_Pred'] = preds_lnn_inv
mse_lnn = mean_squared_error(actual, preds_lnn_inv)
results['LNN'] = {'RMSE': np.sqrt(mse_lnn), 'MSE': mse_lnn, 'R-squared': r2_score(actual, preds_lnn_inv), 'MAE': mean_absolute_error(actual, preds_lnn_inv)}

results_df = pd.DataFrame(results).T
print("Model Performance Metrics:")
print(results_df)

## 6. Advanced LNN Experiments

This section will contain the implementation of the 10 advanced experiments to further explore the capabilities of LNNs.

### 6.1. Experiment 1: Building the LNN Inflow Benchmark

To facilitate future research and ensure reproducibility, we will structure our dataset into a format suitable for sharing on platforms like Hugging Face. This involves creating clear train, validation, and test splits.

**Note:** For this experiment, we define a function that would save the data. The actual file saving is commented out to prevent writing to the local filesystem during this automated run.

In [ ]:
def prepare_huggingface_dataset(df, train_ratio=0.7, val_ratio=0.15):
    # We respect the water-year (Oct-Sep), but for simplicity here, we'll do a standard time-based split
    train_end = int(len(df) * train_ratio)
    val_end = int(len(df) * (train_ratio + val_ratio))

    train_df = df.iloc[:train_end]
    val_df = df.iloc[train_end:val_end]
    test_df = df.iloc[val_end:]

    print(f"Training set size: {len(train_df)}")
    print(f"Validation set size: {len(val_df)}")
    print(f"Test set size: {len(test_df)}")

    # To save for Hugging Face (example):
    # train_df.to_csv('train_inflow_data.csv')
    # val_df.to_csv('validation_inflow_data.csv')
    # test_df.to_csv('test_inflow_data.csv')
    
    return train_df, val_df, test_df

train_split, val_split, test_split = prepare_huggingface_dataset(df.reset_index())

### 6.2. Experiment 2: Irregular-Time Challenge

LNNs are theoretically well-suited for irregularly sampled data. We test this by re-sampling the data, keeping only days with rainfall > 5mm, and training the LNN on this non-uniform grid.

In [ ]:
rainfall_cols = [col for col in df.columns if col.startswith('1')]
df_irregular = df[df[rainfall_cols].sum(axis=1) > 5].copy()

print(f"Original dataset size: {len(df)}")
print(f"Irregular dataset size: {len(df_irregular)}")

# Preprocess and create a generator for the irregular data
data_irregular_scaled = scaler.transform(df_irregular[df.columns[0:]].values)
features_irr = data_irregular_scaled
target_irr = data_irregular_scaled[:, 0]

X_irr, y_irr = create_sequences(features_irr, 0, win_length)
X_irr_t = torch.from_numpy(X_irr).float()
y_irr_t = torch.from_numpy(y_irr).float()

irr_train_loader = DataLoader(TensorDataset(X_irr_t, y_irr_t), batch_size=batch_size, shuffle=False)

# Train a new LNN on the irregular data
lnn_irr_model = TrueLNN(num_features, 128, 1)
print("\nTraining LNN on irregular data:")
lnn_irr_model = train_model(lnn_irr_model, irr_train_loader, epochs=50)

### 6.3. Experiment 3: Spatial Ablation via 'Liquid Connectivity'

We treat the 22 rain gauges as a graph and allow the LNN to learn an adaptive time constant for each, effectively learning the importance of each gauge. We then prune gauges with high time constants (low relevance).

In [ ]:
class SpatiallyAwareLNN(TrueLNN):
    def __init__(self, input_size, hidden_size, output_size):
        super(SpatiallyAwareLNN, self).__init__(input_size, hidden_size, output_size)
        self.time_constants = nn.Parameter(torch.randn(input_size))

    def forward(self, x):
        batch_size = x.size(0)
        h_t = torch.zeros(batch_size, self.hidden_size)
        
        for t in range(x.size(1)):
            # Apply learnable time constants to input features
            input_t = self.input_to_hidden(x[:, t, :] * torch.sigmoid(self.time_constants))
            h_t = odeint(self.ode_func, h_t + input_t, torch.tensor([0, 1]), method='rk4')[1]
            
        out = self.fc(h_t)
        return out

# Initialize and train the spatially aware LNN
lnn_spatial_model = SpatiallyAwareLNN(num_features, 128, 1)
print("\nTraining Spatially Aware LNN:")
lnn_spatial_model = train_model(lnn_spatial_model, train_loader, epochs=50)

# Prune and visualize the learned time constants
learned_tau = torch.sigmoid(lnn_spatial_model.time_constants).detach().numpy()
pruning_threshold = 0.1 # Prune if learned weight is less than 0.1

relevant_gauges = [df.columns[i] for i, tau in enumerate(learned_tau) if tau > pruning_threshold]
print(f"\nRelevant Gauges (learned weight > {pruning_threshold}):")
print(relevant_gauges)

### 6.4. Experiment 4: Uncertainty-Aware LNN (UA-LNN) for Flood-Risk

We implement Monte Carlo (MC) dropout within the LNN to capture uncertainty. This involves adding dropout layers and running multiple forward passes during inference to generate a distribution of predictions.

In [ ]:
class UA_LNN(TrueLNN):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.3):
        super(UA_LNN, self).__init__(input_size, hidden_size, output_size)
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        batch_size = x.size(0)
        h_t = torch.zeros(batch_size, self.hidden_size)
        
        for t in range(x.size(1)):
            input_t = self.input_to_hidden(x[:, t, :])
            # Apply dropout within the ODE solver step
            h_t = odeint(self.ode_func, h_t + self.dropout(input_t), torch.tensor([0, 1]), method='rk4')[1]
            
        out = self.fc(h_t)
        return out

# Initialize and train the UA-LNN
ua_lnn_model = UA_LNN(num_features, 128, 1)
optimizer_ua = optim.Adam(ua_lnn_model.parameters(), lr=0.001)
print("\nTraining Uncertainty-Aware LNN:")
ua_lnn_model = train_model(ua_lnn_model, train_loader, epochs=50)

# Generate prediction bands with MC dropout
def get_prediction_bands(model, data_loader, num_passes=30):
    model.train() # Enable dropout during inference
    predictions = []
    with torch.no_grad():
        for _ in range(num_passes):
            pass_preds = []
            for inputs, _ in data_loader:
                pass_preds.append(model(inputs))
            predictions.append(torch.cat(pass_preds, dim=0))
    predictions = torch.stack(predictions).numpy()
    mean_preds = predictions.mean(axis=0)
    lower_bound = np.percentile(predictions, 2.5, axis=0)
    upper_bound = np.percentile(predictions, 97.5, axis=0)
    return mean_preds, lower_bound, upper_bound

mean_preds, lower_b, upper_b = get_prediction_bands(ua_lnn_model, test_loader)

### 6.5. Experiment 5: 'Cold-Start' Generalization

We simulate a 'cold-start' scenario by training on normal years and fine-tuning on strong/weak monsoon years. This tests the LNN's ability to adapt to new conditions after deployment.

In [ ]:
# Placeholder for IMD indices - assuming we have a 'monsoon_type' column
# For now, we'll create a mock split
df_mock = df.copy()
df_mock['monsoon_type'] = np.random.choice(['normal', 'strong', 'weak'], size=len(df_mock))

normal_years_df = df_mock[df_mock['monsoon_type'] == 'normal']
strong_weak_df = df_mock[df_mock['monsoon_type'] != 'normal']

# This section would involve training on 'normal_years_df' and then fine-tuning
# on 'strong_weak_df' and measuring regret. The implementation is omitted for brevity.

### 6.6. Experiment 6: 1-step vs. K-step Rollout

We test the model's long-term forecasting stability by performing a k-step rollout (14-day forecast) and injecting noise into the inputs.

In [ ]:
def k_step_rollout_pytorch(model, initial_window, k, noise_type=None):
    model.eval()
    current_window = initial_window.clone().detach()
    predictions = []

    for _ in range(k):
        # Add noise if specified
        if noise_type == 'additive':
            current_window += torch.randn_like(current_window) * 0.05
        elif noise_type == 'undercatch':
            current_window[:, 1:] *= 0.9 # -10% under-catch for rainfall

        with torch.no_grad():
            pred = model(current_window.unsqueeze(0))[:, 0]
        predictions.append(pred.item())

        # Roll the window forward
        new_row = current_window[-1, :].clone().detach()
        new_row[0] = pred
        current_window = torch.roll(current_window, -1, dims=0)
        current_window[-1, :] = new_row
        
    return np.array(predictions)

# Example usage (conceptual)
initial_sequence_t = X_test_t[0]
rollout_preds_lnn = k_step_rollout_pytorch(lnn_model, initial_sequence_t, k=14, noise_type='additive')

print(f"14-day LNN rollout predictions (with additive noise):\n{rollout_preds_lnn}")

### 6.7. Experiment 7: 'Physics-infused τ' – Priming the Liquid Constant

Instead of random initialization, we prime the LNN's time constants (`τ`) with physics-based estimates of catchment travel time. This experiment investigates if this prior knowledge improves convergence and model plausibility.

In [ ]:
# Placeholder for physics-based travel times (e.g., from SCS-CN method)
physics_travel_times = np.random.rand(num_features) # Replace with actual estimates
physics_travel_times_tensor = torch.tensor(physics_travel_times, dtype=torch.float32)

class PhysicsLNN(SpatiallyAwareLNN):
    def __init__(self, input_size, hidden_size, output_size, initial_taus):
        super(PhysicsLNN, self).__init__(input_size, hidden_size, output_size)
        # Initialize time constants with physics-based estimates
        self.time_constants.data = initial_taus

# Initialize and train the physics-infused LNN
lnn_physics_model = PhysicsLNN(num_features, 128, 1, physics_travel_times_tensor)
optimizer_physics = optim.Adam(lnn_physics_model.parameters(), lr=0.001)
print("\nTraining Physics-Infused LNN:")
lnn_physics_model = train_model(lnn_physics_model, train_loader, epochs=50)

### 6.8. Experiment 8: Extreme-Value Focus

This experiment focuses on the model's ability to predict extreme inflow events. We use a Peaks-Over-Threshold (POT) approach to identify these events and evaluate the model's performance on the tail of the distribution.

In [ ]:
# Identify extreme events using Peaks-Over-Threshold (POT)
threshold = df_final['Inflows in last 24 Hrs. in Mcft.'].quantile(0.95)
extreme_events_actual = df_final[df_final['Inflows in last 24 Hrs. in Mcft.'] > threshold]

# Evaluate model performance on these extreme events
extreme_preds_lstm = df_final.loc[extreme_events_actual.index, 'LSTM_Pred']
rmse_extreme = np.sqrt(mean_squared_error(extreme_events_actual['Inflows in last 24 Hrs. in Mcft.'], extreme_preds_lstm))

print(f"RMSE on Top 5% Extreme Events (LSTM): {rmse_extreme:.4f}")

### 6.9. Experiment 9: Compute & Carbon Audit

We log the training time and parameter count for each model to provide an audit of computational and potential energy consumption.

In [ ]:
compute_audit = {}

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# --- LSTM ---
start_time = time.time()
model_lstm.fit(train_generator, epochs=1, validation_data=test_generator, shuffle=False, verbose=0)
lstm_time = time.time() - start_time
compute_audit['LSTM'] = {'Training Time (s/epoch)': lstm_time, 'Parameters': model_lstm.count_params()}

# --- LNN ---
start_time = time.time()
train_lnn(lnn_model, train_data_loader, criterion, optimizer, epochs=1)
lnn_time = time.time() - start_time
compute_audit['LNN'] = {'Training Time (s/epoch)': lnn_time, 'Parameters': count_parameters(lnn_model)}

audit_df = pd.DataFrame(compute_audit).T
print("Compute & Carbon Audit:")
print(audit_df)

### 6.10. Experiment 10: Reproducibility Checklist

To ensure this study is fully reproducible, we provide the following components:

- **Raw & Pre-processed Data:** The `final_inflow_rainfall_merged.csv` file is the primary data source.
- **Train/Val/Test Splits:** The exact splits can be reproduced using the `train_test_split` function with `random_state=123` and `shuffle=False`.
- **Hyperparameters:** All hyperparameters (window length, batch size, learning rate, etc.) are explicitly defined in the code cells.
- **Random Seeds:** We set `random_state=123` for the data split and will use fixed seeds for model initializations where applicable.
- **Metrics Notebook:** This notebook itself serves as the metrics notebook, containing all code for evaluation.

In [ ]:
reproducibility_checklist = {
    "Raw Data": "final_inflow_rainfall_merged.csv",
    "Train/Test Split": "test_size=0.20, random_state=123, shuffle=False",
    "Hyperparameters": {
        "window_length": 5,
        "batch_size": 32,
        "epochs": 50,
        "learning_rate": 0.001
    },
    "Random Seeds": "random_state=123 for data split",
    "Metrics Notebook": "This notebook"
}

import json
print("Reproducibility Checklist:")
print(json.dumps(reproducibility_checklist, indent=2))

## 7. Final Conclusion and Comparison

This section synthesizes the results from all baseline models and the 10 LNN experiments. We provide a comprehensive comparison based on both quantitative metrics and qualitative characteristics.

### 7.1. Master Quantitative Comparison

In [ ]:
# This cell will be populated with the final results from all experiments
final_results = results_df.copy()
# ... add results from LNN experiments ...
display(Markdown('**Master Performance Metrics Table**'))
display(final_results)

### 7.2. Qualitative Model Comparison

In [ ]:
qualitative_data = {
    'Metric': ['Interpretability', 'Irregular Data Handling', 'Noise Robustness', 'Uncertainty Quantification', 'Computational Efficiency'],
    'LSTM/GRU': ['Low (Black Box)', 'Requires Imputation', 'Moderate', 'Requires Extensions (e.g., Bayesian)', 'Low (High Params)'],
    '1D-CNN': ['Moderate (Filters)', 'Requires Imputation', 'High', 'Requires Extensions', 'High'],
    'LNN': ['High (τ, Connectivity)', 'Native Support', 'High (Expected)', 'Native (MC Dropout)', 'High (Fewer Params)']
}
qualitative_df = pd.DataFrame(qualitative_data).set_index('Metric')
display(Markdown('**Qualitative Model Comparison**'))
display(qualitative_df)

### 7.3. Summary of Key Findings

This section will provide a narrative summary of the results, drawing conclusions from the tables and visualizations above. It will focus on answering the core research question: **How do LNNs perform in real-world hydrological forecasting compared to other models, and in what specific scenarios do they excel?**

**Key discussion points will include:**

- **Overall Performance:** How did the baseline LNN compare to LSTM, GRU, and CNNs on standard metrics?
- **Irregular Data:** Did the LNN show a distinct advantage when trained on the non-uniform, event-based data grid?
- **Interpretability:** What insights did the spatial ablation and physics-infused `τ` experiments provide about the underlying hydrological system?
- **Robustness and Uncertainty:** How well did the LNN handle noisy inputs, and did the UA-LNN provide meaningful uncertainty estimates for flood-risk assessment?
- **Adaptability:** Did the 'cold-start' experiment suggest that LNNs can adapt to changing conditions more efficiently than traditional models?
- **Efficiency:** Was the LNN more computationally efficient, as suggested by the compute audit?